In [3]:
import pandas as pd

In [4]:
data = pd.read_csv("D:\\P\\python_机器学习\\数据集\\facebook-v-predicting-check-ins\\train\\train.csv")

In [5]:
data.head()

,row_id,x,y,accuracy,time,place_id
0,0,0.7941,9.0809,54,470702,8523065625
1,1,5.9567,4.7968,13,186555,1757726713
2,2,8.3078,7.0407,74,322648,1137537235
3,3,7.3665,2.5165,65,704587,6567393236
4,4,4.0961,1.1307,31,472130,7440663949


In [6]:
data = data.query("x < 2.5 & x > 2 & y < 1.5 & y > 1.0")

In [7]:
data.head()

,row_id,x,y,accuracy,time,place_id
112,112,2.2360,1.3655,66,623174,7663031065
180,180,2.2003,1.2541,65,610195,2358558474
367,367,2.4108,1.3213,74,579667,6644108708
874,874,2.0822,1.1973,320,143566,3229876087
1022,1022,2.0160,1.1659,65,207993,3244363975


In [8]:
time_value = pd.to_datetime(data['time'], unit="s")
date = pd.DatetimeIndex(time_value)

In [9]:
date

DatetimeIndex(['1970-01-08 05:06:14', '1970-01-08 01:29:55',
               '1970-01-07 17:01:07', '1970-01-02 15:52:46',
               '1970-01-03 09:46:33', '1970-01-06 19:49:38',
               '1970-01-06 13:33:24', '1970-01-02 22:49:55',
               '1970-01-04 14:30:10', '1970-01-07 16:57:44',
               ...
               '1970-01-02 09:24:50', '1970-01-01 10:29:34',
               '1970-01-09 11:38:46', '1970-01-02 03:42:14',
               '1970-01-04 22:02:44', '1970-01-09 08:31:25',
               '1970-01-07 12:29:49', '1970-01-09 20:46:26',
               '1970-01-02 18:11:58', '1970-01-01 22:06:09'],
              dtype='datetime64[ns]', name='time', length=83197, freq=None)

In [10]:
data["day"] = date.day
data["weekday"] = date.weekday
data["hour"] = date.hour

In [12]:
data.head()

,row_id,x,y,accuracy,time,place_id,day,weekday,hour
112,112,2.2360,1.3655,66,623174,7663031065,8,3,5
180,180,2.2003,1.2541,65,610195,2358558474,8,3,1
367,367,2.4108,1.3213,74,579667,6644108708,7,2,17
874,874,2.0822,1.1973,320,143566,3229876087,2,4,15
1022,1022,2.0160,1.1659,65,207993,3244363975,3,5,9


In [13]:
#签到次数
place_count = data.groupby("place_id").count()["row_id"]

In [14]:
place_count[place_count > 3]

place_id
1014605271     28
1015645743      4
1017236154     31
1024951487      5
1028119817      4
             ... 
9936666116    140
9954155328      8
9980625005     16
9994257798     25
9996671132     18
Name: row_id, Length: 950, dtype: int64

In [15]:
data_final = data[data["place_id"].isin(place_count[place_count>3].index.values)]

In [16]:
data_final.head()

,row_id,x,y,accuracy,time,place_id,day,weekday,hour
112,112,2.2360,1.3655,66,623174,7663031065,8,3,5
367,367,2.4108,1.3213,74,579667,6644108708,7,2,17
874,874,2.0822,1.1973,320,143566,3229876087,2,4,15
1022,1022,2.0160,1.1659,65,207993,3244363975,3,5,9
1045,1045,2.3859,1.1660,498,503378,6438240873,6,1,19


In [17]:
# 筛选特征值和目标值
x = data_final[["x","y","accuracy","day","weekday","hour"]]
y = data_final["place_id"]

In [18]:
x.head()

,x,y,accuracy,day,weekday,hour
112,2.2360,1.3655,66,8,3,5
367,2.4108,1.3213,74,7,2,17
874,2.0822,1.1973,320,2,4,15
1022,2.0160,1.1659,65,3,5,9
1045,2.3859,1.1660,498,6,1,19


In [19]:
y.head()

112     7663031065
367     6644108708
874     3229876087
1022    3244363975
1045    6438240873
Name: place_id, dtype: int64

In [20]:
# 数据集划分
from sklearn.model_selection import train_test_split

In [21]:
x_train, x_test, y_train, y_test = train_test_split(x,y)

In [22]:
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

In [24]:
# 标准化
transfer = StandardScaler()
x_train = transfer.fit_transform(x_train)
x_test = transfer.transform(x_test)

#KNN算法
estimator = KNeighborsClassifier()

param_dict = {"n_neighbors": [1,3,5,7,9]}
estimator = GridSearchCV(estimator, param_grid=param_dict, cv=3)
estimator.fit(x_train, y_train)

y_predict = estimator.predict(x_test)
print("预测值y_predict: \n", y_predict)
print("比对真实值和预测值: ", y_predict == y_test)

D:\P\python\Lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


预测值y_predict: 
 [1327075245 2209293971 8655245424 ... 8790355618 7240847236 4020469575]
比对真实值和预测值:  5457777     False
951490       True
28452705    False
23107804     True
8807897     False
            ...  
26976112    False
3305178      True
5423175      True
23980556     True
20054086    False
Name: place_id, Length: 20228, dtype: bool


In [27]:
print("准确率：\n", estimator.score(x_test, y_test))
print("最佳参数：\n", estimator.best_params_)
print("最佳结果：\n", estimator.best_score_)
print("最佳估计器：\n", estimator.best_estimator_)
print("交叉验证结果: \n", estimator.cv_results_)

准确率：
 0.35900731659086416
最佳参数：
 {'n_neighbors': 1}
最佳结果：
 0.3386176684688766
最佳估计器：
 KNeighborsClassifier(n_neighbors=1)
交叉验证结果: 
 {'mean_fit_time': array([0.07362278, 0.06920417, 0.07048416, 0.07384793, 0.07331689]), 'std_fit_time': array([0.00516214, 0.00236861, 0.00247204, 0.00337788, 0.00184414]), 'mean_score_time': array([0.53900806, 0.63622069, 0.70047331, 0.74339771, 0.8008194 ]), 'std_score_time': array([0.01507732, 0.02606975, 0.00624347, 0.02358544, 0.00847205]), 'param_n_neighbors': masked_array(data=[1, 3, 5, 7, 9],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'params': [{'n_neighbors': 1}, {'n_neighbors': 3}, {'n_neighbors': 5}, {'n_neighbors': 7}, {'n_neighbors': 9}], 'split0_test_score': array([0.34125964, 0.32835673, 0.33834289, 0.33745304, 0.33414079]), 'split1_test_score': array([0.33712365, 0.32031443, 0.33608543, 0.3382113 , 0.33717309]), 'split2_test_score': array([0.33746972, 0.31947397, 0.33064715, 0.33